In [1]:
TRAIN_CSV_PATH = r"../datasets/train.csv"
PRICES_CSV_PATH = r"../datasets/prices.csv"
CAL_CSV_PATH = r"../datasets/calendar.csv"
SAMPLE_CSV_PATH = r"../datasets/sample_submission.csv"

EXPORT_PATH = r"../submissions/"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import Naive, SeasonalNaive, WindowAverage, SeasonalWindowAverage, SimpleExponentialSmoothingOptimized, SeasonalExponentialSmoothingOptimized, Holt, HoltWinters

from sklearn.metrics import mean_squared_error

c:\Users\Tristan\Desktop\Assignments\ADS\ads-final-project\ads_env\lib\site-packages\statsforecast\core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
df_train = pd.read_csv(TRAIN_CSV_PATH)
df_sample_sub = pd.read_csv(SAMPLE_CSV_PATH)
df_cal = pd.read_csv(CAL_CSV_PATH)

df_dates = pd.DataFrame(columns = ["d"], data = df_train.columns[6:])
df_dates = df_dates.merge(df_cal[["date", "d"]], on = "d", how = "left")

In [14]:
# Function to convert predictions into submission csv
def convert_to_sub_csv(preds_df, method) :
    df_converted = preds_df[["unique_id", "ds", method]].pivot(index = "unique_id", columns = "ds", values = method)

    # Change col names back to day ints
    day_to_d = dict(zip(list(df_converted.columns), list(df_sample_sub.columns[1:])))
    df_converted = df_converted.rename(day_to_d, axis = 1).reset_index()

    # Round up to nearest int
    df_converted.iloc[:, 1:] = df_converted.iloc[:, 1:].round().astype(int)

    # Sort into the original ordering by ID
    df_converted[["category", "store", "num", "region", "num_2"]] = df_converted["unique_id"].str.split("_", expand = True)
    df_converted["region"] = pd.Categorical(df_converted["region"], ["East", "Central", "West"])
    df_converted = df_converted.sort_values(by = ["region", "num_2", "category", "store", "num"])
    df_converted = df_converted.drop(["category", "store", "num", "region", "num_2"], axis =1)

    # Rename ID col
    df_converted = df_converted.rename(columns = {"unique_id" : "id"})

    return df_converted

In [4]:
# Convert to appropriate datatypes
df_train["id"] = df_train["id"].astype("string")
df_train["item_id"] = df_train["item_id"].astype("string")
df_train["subcat_id"] = df_train["subcat_id"].astype("string")
df_train["category_id"] = df_train["category_id"].astype("string")
df_train["store_id"] = df_train["store_id"].astype("string")
df_train["region_id"] = df_train["region_id"].astype("string")


# Change dtypes
df_cal["date"] = pd.to_datetime(df_cal["date"])
df_cal["weekday"] = df_cal["weekday"].astype("string")
df_cal["d"] = df_cal["d"].astype("string")
df_cal["wm_yr_wk"] = df_cal["wm_yr_wk"].astype(int)

# Add column with months in string
month_names_ls = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
month_int_ls = list(range(1, 13))
month_names_dict = {month_int_ls[i]: month_names_ls[i] for i in range(len(month_int_ls))}

df_cal["month_name"] = df_cal["month"].replace(month_names_dict)

In [5]:
df_dates["d"] = df_dates["d"].astype("string")
df_dates["date"] = pd.to_datetime(df_dates["date"])

#### Super Naive models (and more):
- Items sold same as previous 21 days

Kaggle score: 2.81

In [4]:
df_sample_sub.iloc[:,1:] = df_train.iloc[:, -21:].values

In [5]:
# df_sample_sub.to_csv(EXPORT_PATH+"/submission_4.csv", header=True, index=False)

- Items sold same as previous 21 days + 1

Kaggle score: 2.95

In [6]:
df_sample_sub.iloc[:,1:] = df_train.iloc[:, -21:].values+1

In [7]:
# df_sample_sub.to_csv(EXPORT_PATH+"/submission_5.csv", header=True, index=False)

### Forecasting using naive methods from statsmodels

From [here](https://forecastegy.com/posts/naive-time-series-forecasting-in-python/#:~:text=the%20older%20ones.-,Seasonal%20Naive%20Forecast,than%20the%20simple%20naive%20model.)

In [6]:
dates_dict = dict(zip(list(df_dates["d"]), list(df_dates["date"])))
df_train_dates = df_train.rename(dates_dict, axis = 1)

df_naive_train = df_train_dates[["id"] + list(df_train_dates.columns[6:])].melt(id_vars = ["id"], var_name= "ds", value_name = "y")
df_naive_train = df_naive_train.rename(columns = {"id":"unique_id"})
df_naive_train.head()

,unique_id,ds,y
0,Beauty_1_001_East_1,2011-01-29,0
1,Beauty_1_002_East_1,2011-01-29,0
2,Beauty_1_003_East_1,2011-01-29,0
3,Beauty_1_004_East_1,2011-01-29,0
4,Beauty_1_005_East_1,2011-01-29,0


#### Testing RMSE on train / test validation set 

With Naive, SeasonalNaive, WindowAverage, Seasonal Window Average
- seasonal length 7 days

In [7]:
# Split into train and validation 
train = df_naive_train.loc[df_naive_train['ds'] < '2015-01-29']
valid = df_naive_train.loc[(df_naive_train['ds'] >= '2015-01-29')]
h = valid['ds'].nunique()

In [18]:
model = StatsForecast(models=[Naive(), 
                              SeasonalNaive(season_length=7), 
                              WindowAverage(window_size=7), 
                              SeasonalWindowAverage(window_size=2, season_length=7)],
                      freq='D', n_jobs=-1)
model.fit(train)

StatsForecast(models=[Naive,SeasonalNaive,WindowAverage,SeasWA])

In [19]:
p = model.predict(h=h)
p = p.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')
p.head()

,unique_id,ds,Naive,SeasonalNaive,WindowAverage,SeasWA,y
0,Beauty_1_001_Central_1,2015-01-29,0.0,0.0,0.0,0.0,1
1,Beauty_1_001_Central_1,2015-01-30,0.0,0.0,0.0,0.0,0
2,Beauty_1_001_Central_1,2015-01-31,0.0,0.0,0.0,0.0,0
3,Beauty_1_001_Central_1,2015-02-01,0.0,0.0,0.0,0.0,1
4,Beauty_1_001_Central_1,2015-02-02,0.0,0.0,0.0,0.0,0


In [21]:
print(np.sqrt(mean_squared_error(valid["y"], p["Naive"])))
print(np.sqrt(mean_squared_error(valid["y"], p["SeasonalNaive"])))
print(np.sqrt(mean_squared_error(valid["y"], p["WindowAverage"])))
print(np.sqrt(mean_squared_error(valid["y"], p["SeasWA"])))

4.60855218075943
4.804250660557115
4.49464505136421
4.702569543222306


### Trying with different seasonal and window lengths
With SeasonalNaive, WindowAverage, Seasonal Window Average
- seasonal length 7 days

In [8]:
model_2 = StatsForecast(models=[SeasonalNaive(season_length=14), 
                              WindowAverage(window_size=14), 
                              SeasonalWindowAverage(window_size=2, season_length=7)],
                      freq='D', n_jobs=-1)
model_2.fit(train)

StatsForecast(models=[SeasonalNaive,WindowAverage,SeasWA])

In [9]:
p_2 = model_2.predict(h=h)
p_2 = p_2.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')
p_2.head()

,unique_id,ds,SeasonalNaive,WindowAverage,SeasWA,y
0,Beauty_1_001_Central_1,2015-01-29,0.0,0.0,0.0,1
1,Beauty_1_001_Central_1,2015-01-30,0.0,0.0,0.0,0
2,Beauty_1_001_Central_1,2015-01-31,0.0,0.0,0.0,0
3,Beauty_1_001_Central_1,2015-02-01,0.0,0.0,0.0,1
4,Beauty_1_001_Central_1,2015-02-02,0.0,0.0,0.0,0


In [11]:
print(np.sqrt(mean_squared_error(valid["y"], p_2["SeasonalNaive"])))
print(np.sqrt(mean_squared_error(valid["y"], p_2["WindowAverage"])))
print(np.sqrt(mean_squared_error(valid["y"], p_2["SeasWA"])))

4.869437341538874
4.522386924418987
4.702569543222306


### Producing 21 day forecast, training on the entire train.csv

In [ ]:
model_full = StatsForecast(df = df_naive_train, 
                        models=[Naive(), 
                              SeasonalNaive(season_length=7), 
                              WindowAverage(window_size=7), 
                              SeasonalWindowAverage(window_size=2, season_length=7)],
                      freq='D', n_jobs=-1)

preds_full = model_full.forecast(h = 21)

In [ ]:
preds_full = preds_full.reset_index()
preds_full.head()

,unique_id,ds,Naive,SeasonalNaive,WindowAverage,SeasWA
0,Beauty_1_001_Central_1,2016-05-01,0.0,2.0,0.571429,1.0
1,Beauty_1_001_Central_1,2016-05-02,0.0,0.0,0.571429,0.0
2,Beauty_1_001_Central_1,2016-05-03,0.0,0.0,0.571429,0.5
3,Beauty_1_001_Central_1,2016-05-04,0.0,0.0,0.571429,0.0
4,Beauty_1_001_Central_1,2016-05-05,0.0,2.0,0.571429,1.5


#### Exporting seasonal window average

In [ ]:
df_seaswa = preds_full[["unique_id", "ds", "SeasWA"]].pivot(index = "unique_id", columns = "ds", values = "SeasWA")

# Change col names back to day ints
day_to_d = dict(zip(list(df_seaswa.columns), list(df_sample_sub.columns[1:])))
df_seaswa = df_seaswa.rename(day_to_d, axis = 1).reset_index()

# Round up to nearest int
df_seaswa.iloc[:, 1:] = df_seaswa.iloc[:, 1:].apply(np.ceil).astype(int)

df_seaswa.head()

ds,unique_id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940
0,Beauty_1_001_Central_1,1,0,1,0,2,0,0,1,0,...,2,0,0,1,0,1,0,2,0,0
1,Beauty_1_001_Central_2,0,0,0,1,0,1,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,Beauty_1_001_Central_3,0,0,1,1,1,0,0,0,0,...,1,0,0,0,0,1,1,1,0,0
3,Beauty_1_001_East_1,1,1,1,1,3,0,2,1,1,...,3,0,2,1,1,1,1,3,0,2
4,Beauty_1_001_East_2,2,1,1,2,1,1,1,2,1,...,1,1,1,2,1,1,2,1,1,1


In [ ]:
# Sort into the original ordering by ID
df_seaswa[["category", "store", "num", "region", "num_2"]] = df_seaswa["unique_id"].str.split("_", expand = True)
df_seaswa["region"] = pd.Categorical(df_seaswa["region"], ["East", "Central", "West"])
df_seaswa = df_seaswa.sort_values(by = ["region", "num_2", "category", "store", "num"])
df_seaswa = df_seaswa.drop(["category", "store", "num", "region", "num_2"], axis =1)

# Rename ID col
df_seaswa = df_seaswa.rename(columns = {"unique_id" : "id"})

df_seaswa.head()

ds,id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940
3,Beauty_1_001_East_1,1,1,1,1,3,0,2,1,1,...,3,0,2,1,1,1,1,3,0,2
13,Beauty_1_002_East_1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
23,Beauty_1_003_East_1,2,1,1,1,1,1,2,2,1,...,1,1,2,2,1,1,1,1,1,2
33,Beauty_1_004_East_1,4,2,1,1,2,4,4,4,2,...,2,4,4,4,2,1,1,2,4,4
43,Beauty_1_005_East_1,3,1,1,2,3,2,1,3,1,...,3,2,1,3,1,1,2,3,2,1


In [ ]:
# df_seaswa.to_csv(EXPORT_PATH+"/submission_6.csv", header=True, index=False)

#### Exporting window average (lowest RMSE according to the train / test split)

In [ ]:
df_wa = preds_full[["unique_id", "ds", "WindowAverage"]].pivot(index = "unique_id", columns = "ds", values = "WindowAverage")
day_to_d = dict(zip(list(df_wa.columns), list(df_sample_sub.columns[1:])))

# Change col names back to original day IDs
df_wa = df_wa.rename(day_to_d, axis = 1).reset_index()

# Round up
df_wa.iloc[:, 1:] = df_wa.iloc[:, 1:].apply(np.ceil).astype(int)

df_wa.head()

ds,unique_id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940
0,Beauty_1_001_Central_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,Beauty_1_001_Central_2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,Beauty_1_001_Central_3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,Beauty_1_001_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,Beauty_1_001_East_2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
# Sort into the original ordering by ID
df_wa[["category", "store", "num", "region", "num_2"]] = df_wa["unique_id"].str.split("_", expand = True)
df_wa["region"] = pd.Categorical(df_wa["region"], ["East", "Central", "West"])
df_wa = df_wa.sort_values(by = ["region", "num_2", "category", "store", "num"])
df_wa = df_wa.drop(["category", "store", "num", "region", "num_2"], axis =1)

# Rename ID col
df_wa = df_wa.rename(columns = {"unique_id" : "id"})

df_wa.head()

ds,id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940
3,Beauty_1_001_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
13,Beauty_1_002_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
23,Beauty_1_003_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
33,Beauty_1_004_East_1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
43,Beauty_1_005_East_1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [ ]:
# df_wa.to_csv(EXPORT_PATH+"/submission_7.csv", header=True, index=False)

#### Exporting window average (rounding to nearest int instead of rounding up)

In [ ]:
df_wa = preds_full[["unique_id", "ds", "WindowAverage"]].pivot(index = "unique_id", columns = "ds", values = "WindowAverage")
day_to_d = dict(zip(list(df_wa.columns), list(df_sample_sub.columns[1:])))

# Change col names back to original day IDs
df_wa = df_wa.rename(day_to_d, axis = 1).reset_index()

# Round up
df_wa.iloc[:, 1:] = df_wa.iloc[:, 1:].round().astype(int)

df_wa.head()

ds,unique_id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940
0,Beauty_1_001_Central_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,Beauty_1_001_Central_2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Beauty_1_001_Central_3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,Beauty_1_001_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,Beauty_1_001_East_2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
# Sort into the original ordering by ID
df_wa[["category", "store", "num", "region", "num_2"]] = df_wa["unique_id"].str.split("_", expand = True)
df_wa["region"] = pd.Categorical(df_wa["region"], ["East", "Central", "West"])
df_wa = df_wa.sort_values(by = ["region", "num_2", "category", "store", "num"])
df_wa = df_wa.drop(["category", "store", "num", "region", "num_2"], axis =1)

# Rename ID col
df_wa = df_wa.rename(columns = {"unique_id" : "id"})

df_wa.head()

ds,id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940
3,Beauty_1_001_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
13,Beauty_1_002_East_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,Beauty_1_003_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
33,Beauty_1_004_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
43,Beauty_1_005_East_1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [ ]:
# df_wa.to_csv(EXPORT_PATH+"/submission_10.csv", header=True, index=False)

#### Exporting naive 

In [ ]:
df_naive = preds_full[["unique_id", "ds", "Naive"]].pivot(index = "unique_id", columns = "ds", values = "Naive")
day_to_d = dict(zip(list(df_naive.columns), list(df_sample_sub.columns[1:])))

# Change col names back to original day IDs
df_naive = df_naive.rename(day_to_d, axis = 1).reset_index()

# Round up
df_naive.iloc[:, 1:] = df_naive.iloc[:, 1:].apply(np.ceil).astype(int)

In [ ]:
# Sort into the original ordering by ID
df_naive[["category", "store", "num", "region", "num_2"]] = df_naive["unique_id"].str.split("_", expand = True)
df_naive["region"] = pd.Categorical(df_naive["region"], ["East", "Central", "West"])
df_naive = df_naive.sort_values(by = ["region", "num_2", "category", "store", "num"])
df_naive = df_naive.drop(["category", "store", "num", "region", "num_2"], axis =1)

# Rename ID col
df_naive = df_naive.rename(columns = {"unique_id" : "id"})

df_naive.head()

ds,id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940
3,Beauty_1_001_East_1,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
13,Beauty_1_002_East_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,Beauty_1_003_East_1,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
33,Beauty_1_004_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
43,Beauty_1_005_East_1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# df_naive.to_csv(EXPORT_PATH+"/submission_8.csv", header=True, index=False)

#### Exporting seasonal naive

In [ ]:
df_s_naive = preds_full[["unique_id", "ds", "SeasonalNaive"]].pivot(index = "unique_id", columns = "ds", values = "SeasonalNaive")
day_to_d = dict(zip(list(df_s_naive.columns), list(df_sample_sub.columns[1:])))

# Change col names back to original day IDs
df_s_naive = df_s_naive.rename(day_to_d, axis = 1).reset_index()

# Round up
df_s_naive.iloc[:, 1:] = df_s_naive.iloc[:, 1:].apply(np.ceil).astype(int)

In [ ]:
# Sort into the original ordering by ID
df_s_naive[["category", "store", "num", "region", "num_2"]] = df_s_naive["unique_id"].str.split("_", expand = True)
df_s_naive["region"] = pd.Categorical(df_s_naive["region"], ["East", "Central", "West"])
df_s_naive = df_s_naive.sort_values(by = ["region", "num_2", "category", "store", "num"])
df_s_naive = df_s_naive.drop(["category", "store", "num", "region", "num_2"], axis =1)

# Rename ID col
df_s_naive = df_s_naive.rename(columns = {"unique_id" : "id"})

df_s_naive.head()

ds,id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940
3,Beauty_1_001_East_1,1,0,0,0,2,0,3,1,0,...,2,0,3,1,0,0,0,2,0,3
13,Beauty_1_002_East_1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
23,Beauty_1_003_East_1,1,0,0,1,1,0,2,1,0,...,1,0,2,1,0,0,1,1,0,2
33,Beauty_1_004_East_1,2,0,0,1,2,4,1,2,0,...,2,4,1,2,0,0,1,2,4,1
43,Beauty_1_005_East_1,4,1,0,2,3,1,0,4,1,...,3,1,0,4,1,0,2,3,1,0


### Trying with ETS models

SimpleExponentialSmoothingOptimized, SeasonalExponentialSmoothingOptimized, Holt, HoltWinters

In [8]:
# Subset the last 50% of the time series because of computation
df_naive_train = df_naive_train.set_index("ds")
df_naive_train = df_naive_train.sort_index()
df_naive_train["ds"] = df_naive_train.index

# Find midpoint
midpoint = df_naive_train["ds"].iloc[0] + ((df_naive_train["ds"].iloc[-1] - df_naive_train["ds"].iloc[0]) / 2)

# Split into train and validation, from midpoint onwards
df_naive_train_subset = df_naive_train.loc[midpoint:]

# find date to split, 80% of subset
test_split_date = ((df_naive_train["ds"].iloc[-1] - midpoint) * 0.8) + midpoint

df_naive_train_subset = df_naive_train_subset.drop("ds", axis = 1)
df_naive_train_subset = df_naive_train_subset.reset_index()

# Split by date
train = df_naive_train_subset.loc[df_naive_train_subset['ds'] < test_split_date]
valid = df_naive_train_subset.loc[(df_naive_train_subset['ds'] >= test_split_date)]

h = valid['ds'].nunique()

In [9]:
model_3 = StatsForecast(models=[SimpleExponentialSmoothingOptimized(), 
                              SeasonalExponentialSmoothingOptimized(season_length=7)
                              ],
                      freq='D', n_jobs=-1)

p_3 = model_3.fit_predict(h = h, df = train)

In [10]:
p_3 = p_3.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')
p_3.head()

,unique_id,ds,SESOpt,SeasESOpt,y
0,Beauty_1_001_Central_1,2015-10-22,0.239553,0.283106,0
1,Beauty_1_001_Central_1,2015-10-23,0.239553,0.352633,0
2,Beauty_1_001_Central_1,2015-10-24,0.239553,0.118980,0
3,Beauty_1_001_Central_1,2015-10-25,0.239553,0.191915,0
4,Beauty_1_001_Central_1,2015-10-26,0.239553,0.224423,0


In [12]:
print(np.sqrt(mean_squared_error(valid["y"], p_3["SESOpt"])))
print(np.sqrt(mean_squared_error(valid["y"], p_3["SeasESOpt"])))

4.529695390799881
4.719031412831159


### Forecasting for the next 21 days using 50% of the train data

In [16]:
p_3_forecast = model_3.forecast(h = 21, df = df_naive_train_subset)

In [17]:
p_3_forecast = p_3_forecast.reset_index()
p_3_forecast.head()

,unique_id,ds,SESOpt,SeasESOpt
0,Beauty_1_001_Central_1,2016-05-01,0.322579,0.211212
1,Beauty_1_001_Central_1,2016-05-02,0.322579,0.445361
2,Beauty_1_001_Central_1,2016-05-03,0.322579,0.136753
3,Beauty_1_001_Central_1,2016-05-04,0.322579,0.205991
4,Beauty_1_001_Central_1,2016-05-05,0.322579,0.279417


In [18]:
df_SEASOpt = convert_to_sub_csv(p_3_forecast, "SESOpt")

In [21]:
df_SeasESOpt = convert_to_sub_csv(p_3_forecast, "SeasESOpt")

In [20]:
# df_SEASOpt.to_csv(EXPORT_PATH+"/submission_11.csv", header=True, index=False)

In [22]:
# df_SeasESOpt.to_csv(EXPORT_PATH+"/submission_12.csv", header=True, index=False)